In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

In [2]:
df_sika_original = pd.read_csv('cdc_zika.csv', low_memory=False)

In [3]:
df = df_sika_original.drop(columns=['time_period','time_period_type'])
df = df.astype('str')
df_codes= pd.read_excel('countries_codes.xlsx')
df_codes.set_index(['Alfa-2'],inplace=True)

In [4]:
df.dtypes

report_date        object
location           object
location_type      object
data_field         object
data_field_code    object
value              object
unit               object
dtype: object

In [5]:
df.columns

Index(['report_date', 'location', 'location_type', 'data_field',
       'data_field_code', 'value', 'unit'],
      dtype='object')

In [6]:
df_codes.columns

Index(['Nombre del país', 'ISO 3166-1', 'Alfa-3'], dtype='object')

In [7]:
df = df[df['report_date'] != 'nan'].copy()
df.reset_index(drop=True,inplace=True)

In [8]:
df['report_date']= df['report_date'].str.replace('[_-]','-')
df['report_date'] = pd.to_datetime(df['report_date'])

In [9]:
df['country']= df['location'].str.split('-').apply(lambda x: ' '.join(x[0].split('_')))

In [10]:
def remove_under_line(x):
    array= x.split('_')
    return ' '.join(array)

In [11]:
df['location']=(
    df['location']
    .apply(lambda x:' '.join(x.split('-')))
    .apply(remove_under_line)
)

In [12]:
df['data_field']= df['data_field'].apply(remove_under_line).str.lower()

In [13]:
def elimina_asteriscos(x):
    if(x.find('*')>0):
        return 0.0
    else:
        return x

In [14]:
df['value']= df['value'].apply(elimina_asteriscos)

In [15]:
df['value']=pd.to_numeric(df['value'],errors='coerce')

In [16]:
def define_country(data_code):
    alpha_code = data_code[:2]
    if(alpha_code == 'HA'):
        alpha_code = 'HT'
    return df_codes.loc[alpha_code]['Nombre del país']

In [17]:
df['country']= df['data_field_code'].apply(define_country)

In [18]:
df.to_csv('zika_cleaded.csv',index=False)

In [19]:
df.columns

Index(['report_date', 'location', 'location_type', 'data_field',
       'data_field_code', 'value', 'unit', 'country'],
      dtype='object')

In [20]:
df['data_field']

0            cumulative confirmed local cases
1             cumulative probable local cases
2         cumulative confirmed imported cases
3          cumulative probable imported cases
4                cumulative cases under study
                         ...                 
107607               confirmed conjunctivitis
107608                      confirmed eyepain
107609                     confirmed headache
107610                      confirmed malaise
107611                       zika no specimen
Name: data_field, Length: 107612, dtype: object

In [21]:
df_filtered= df[~df['data_field'].str.contains('not|no')]
df_filtered.reset_index(drop=True,inplace=True)
df_filtered.head(5)

,report_date,location,location_type,data_field,data_field_code,value,unit,country
0,2016-03-19,Argentina Buenos Aires,province,cumulative confirmed local cases,AR0001,0.0,cases,Argentina
1,2016-03-19,Argentina Buenos Aires,province,cumulative probable local cases,AR0002,0.0,cases,Argentina
2,2016-03-19,Argentina Buenos Aires,province,cumulative confirmed imported cases,AR0003,2.0,cases,Argentina
3,2016-03-19,Argentina Buenos Aires,province,cumulative probable imported cases,AR0004,1.0,cases,Argentina
4,2016-03-19,Argentina Buenos Aires,province,cumulative cases under study,AR0005,127.0,cases,Argentina


In [22]:
substrings=[
    'rash',
    'arthritis',
    'arthralgia',
    'fever',
    'conjunctivitis',
    'eyepain',
    'headache',
    'malaise',
    'male',
    'local',
    'discarded',
    'microcephaly',
    'suspected',
    'study',
    'gbs',
    'pending',
    'efe',
    'hospitalized',
    '201'
] 
(df_filtered[~df_filtered['data_field']
             .str.contains('|'.join(substrings))]['data_field']
             .unique()
)

array(['cumulative confirmed imported cases',
       'cumulative probable imported cases', 'zika reported',
       'zika confirmed laboratory', 'zika confirmed clinic',
       'total zika new confirmed pcr', 'zika new confirmed pcr f',
       'zika new confirmed pcr m', 'zika confirmed pcr cumulative',
       'zika confirmed pregnant cumulative',
       'total zika confirmed cumulative', 'total zika confirmed imported',
       'total zika confirmed pregnant',
       'total zika confirmed ages 0-11mo f',
       'total zika confirmed ages 0-11mo m',
       'total zika confirmed ages 1-4yrs f',
       'total zika confirmed ages 1-4yrs m',
       'total zika confirmed ages 5-9yrs f',
       'total zika confirmed ages 5-9yrs m',
       'total zika confirmed ages 10-14yrs f',
       'total zika confirmed ages 10-14yrs m',
       'total zika confirmed ages 15-19yrs f',
       'total zika confirmed ages 15-19yrs m',
       'total zika confirmed ages 20-49yrs f',
       'total zika confirmed ag

In [23]:
df_filtered=(df_filtered[~df_filtered['data_field'].str.contains('|'.join(substrings))])

In [25]:
zika_countries=df_filtered.groupby(['country'])['value'].sum().sort_values()
zika_countries

country
Argentina                                              378.0
Panamá                                                 408.0
El Salvador                                            472.0
México                                                 565.0
Guatemala                                             1578.0
República Dominicana                                  2376.0
Nicaragua                                             4669.0
Puerto Rico                                           5579.0
Ecuador                                               5839.0
Islas Vírgenes de los Estados Unidos de América       8089.0
Estados Unidos de América                             8933.0
Colombia                                            443745.0
Brasil                                             2777196.0
Name: value, dtype: float64

In [43]:
df_filtered_brazil= df_filtered[df_filtered['country'] == 'Brasil']
zika_locations_brazil= df_filtered_brazil.groupby('location')['value'].sum().sort_values()
zika_locations_brazil

location
Brazil Roraima                   481.0
Brazil Santa Catarina            630.0
Brazil Piaui                     719.0
Brazil Amapa                     918.0
Brazil Rio Grande do Sul        2075.0
Brazil Distrito Federal         2152.0
Brazil Pernambuco               3031.0
Brazil Sergipe                  3162.0
Brazil Mato Grosso do Sul       4023.0
Brazil Acre                     5235.0
Brazil Rondonia                 6110.0
Brazil Ceara                    6820.0
Brazil Para                     9438.0
Brazil Rio Grande do Norte     11344.0
Brazil Espirito Santo          13138.0
Brazil Parana                  13448.0
Brazil Maranhao                13643.0
Brazil Amazonas                14556.0
Brazil Paraiba                 15497.0
Sul                            16153.0
Brazil Alagoas                 16896.0
Brazil Goias                   17484.0
Brazil Tocantins               20041.0
Brazil Sao Paulo               22958.0
Norte                          56779.0
Brazil Minas Ger

In [44]:
df_filtered.groupby('report_date')['value'].sum()

report_date
2015-11-28       2.0
2015-12-05       0.0
2015-12-09      58.0
2015-12-12       0.0
2015-12-16      58.0
               ...  
2016-06-25    8613.0
2016-06-26      19.0
2016-06-28     739.0
2016-06-29    2386.0
2016-07-01     595.0
Name: value, Length: 115, dtype: float64